This notebook demonstrate the usage of this class

In [1]:
from mloversampler import MLOverSampling
from sqlalchemy import create_engine
import pandas as pd

## Load data and split into X and y datasets

In [2]:
# Load your data here
engine = create_engine('sqlite:///../data/DisasterResponse.db')
conn = engine.connect()
df = pd.read_sql('select * from messages', con=conn, index_col='id')
df.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# create X and y datasets
y = df.iloc[:, 2:]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# list of all the classes
list(y.columns)

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [5]:
X = df[['message']]

## Instantiate the class

In [6]:
ml = MLOverSampling()

## Demonstrating usage of tail_labels
This function return a list of the minority samples with mean in 20% percentile

In [34]:
tail_labels = ml.get_tail_labels(y)
tail_labels

['offer',
 'security',
 'clothing',
 'missing_people',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'fire']

## Demonstrating usage of sample_ratio's
This function return a series with the factor indicating level of imbalance of each class
Classes that are not underrepresented have a factor 1 by default

In [8]:
sample_ratios = ml.get_sample_ratio(y)
sample_ratios

offer                     21.0
shops                     20.0
tools                     15.0
hospitals                  9.0
fire                       9.0
missing_people             8.0
aid_centers                8.0
security                   6.0
clothing                   6.0
related                    1.0
other_infrastructure       1.0
floods                     1.0
weather_related            1.0
buildings                  1.0
storm                      1.0
earthquake                 1.0
cold                       1.0
other_weather              1.0
electricity                1.0
other_aid                  1.0
transport                  1.0
infrastructure_related     1.0
request                    1.0
death                      1.0
refugees                   1.0
money                      1.0
shelter                    1.0
food                       1.0
water                      1.0
military                   1.0
search_and_rescue          1.0
medical_products           1.0
medical_

In [83]:
sample_ratios.iloc[0]

21.0

## Demonstrating usage of minority_oversampling
Duplicate each record in minority classes with the calculated ratio
e.g. class 'offer' are duplicated 21 times, whilst class 'related' with ratio = 1 are not duplicated

In [38]:
X_new, y_new, tail_labels = ml.minority_oversampling(X, y)
print('X and y shape before: {} - {}'.format(X.shape, y.shape))
print('X and y Shape after: {} - {}'.format(X_new.shape, y_new.shape))
      

Imbalanced labels: ['offer', 'security', 'clothing', 'missing_people', 'tools', 'hospitals', 'shops', 'aid_centers', 'fire']
Minority samples: (2069, 1) (2069, 35)
X and y shape before: (26179, 1) - (26179, 35)
X and y Shape after: (45977, 1) - (45977, 35)


In [86]:
# minority class duplication
minority_records_before = len(y[y['offer'] == 1])
minority_records_after = len(y_new[y_new['offer'] == 1])
print('Label `OFFER`: Before {}, After {}'.format(minority_records_before, minority_records_after))

Label `OFFER`: Before 118, After 2596


In [87]:
# over represented class duplication
minority_records_before = len(y[y['related'] == 1])
minority_records_after = len(y_new[y_new['related'] == 1])
print('Label `RELATED`: Before {}, After {}'.format(minority_records_before, minority_records_after))

Label `RELATED`: Before 19876, After 39674
